In [ ]:
import sqlite3 as sql
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import yaml
import scipy
import os 

import MopsAnalysis
import MopsReader
import MopsPlotter
from MopsTracker import MopsTracker
from MopsParameters import MopsParameters

% matplotlib inline

In [ ]:
dataLocation = "/Users/joachim/repos/neosim/unittest/testData/testsources.db"
subsampleLocation = "/Users/joachim/repos/neosim/unittest/testData/test/"

In [ ]:
con = sql.connect(dataLocation)

In [ ]:
full = pd.read_sql_query("""
SELECT * FROM testsources
""", con)

In [ ]:
MopsPlotter.plotDataframe(full)

In [ ]:
ssmids = pd.read_sql_query("""
SELECT DISTINCT ssmid FROM testsources
""", con)

In [ ]:
len(ssmids['ssmid'])

In [ ]:
obj_num = 3

In [ ]:
num_ssm = random.sample(ssmids['ssmid'], obj_num)
sample = ""
for i in num_ssm:
    sample += str(i) + ', '
    
sample = '(' + sample[0:-2] + ')'

In [ ]:
objs = pd.read_sql_query("""
SELECT * FROM testsources
WHERE ssmid IN %s
""" % (sample), con, index_col='diaid')

In [ ]:
objs

In [ ]:
subsampleData = subsampleLocation + "subsampledata.txt"

In [ ]:
objs.to_csv(subsampleData, sep=" ", header=False, index='diaid')

In [ ]:
MopsPlotter.plotDataframe(objs)

In [ ]:
! rm -rf {subsampleLocation}/nightly/
! rm -rf {subsampleLocation}/obshist/
! mkdir {subsampleLocation}/nightly
! mkdir {subsampleLocation}/obshist

nightly = subsampleLocation + "/nightly"
obshist = subsampleLocation + "/obshist"

In [ ]:
import subprocess
call = ["python", os.getenv("MOPS_DIR") + "/bin/splitByNight.py", "-n", nightly,subsampleData]
subprocess.call(call);

In [ ]:
! rm -rf {subsampleLocation}/subsampleRun

In [ ]:
! python runMops.py {subsampleLocation}/nightly {subsampleLocation}/subsampleRun

In [ ]:
parameters = MopsParameters.fromYaml(subsampleLocation + "subsampleRun/parameters.yaml")
tracker = MopsTracker.fromYaml(subsampleLocation + "subsampleRun/tracker.yaml")

In [ ]:
MopsPlotter.plotTracklets(tracker.diasources[:1], tracker.tracklets[:1])

In [ ]:
MopsPlotter.plotTracklets(tracker.diasources, tracker.collapsedTrackletsById)

In [ ]:
MopsPlotter.plotTracklets(tracker.diasources, tracker.purifiedTrackletsById)

In [ ]:
MopsPlotter.plotTracks(tracker.dets, tracker.tracks)

In [ ]:
tracker.finalTracks

In [ ]:
MopsPlotter.plotTracks(tracker.dets, tracker.finalTracks)

In [ ]:
import MopsAnalysis

In [ ]:
analysis = MopsAnalysis.runAnalysis(parameters, tracker)

In [ ]:
analysis.analyze()

In [ ]:
analysis.finalTrackletDetFileSizes

In [ ]:
analysis.totalTracklets

In [ ]:
analysis.totalCollapsedTracklets

In [ ]:
analysis.subsetTracks

In [ ]:
analysis.longestTracks

In [ ]:
analysis.ssmidsOfInterest

In [ ]:
analysis.ssmidsOfInterestObjects[2].tracks

In [ ]:
testObject = analysis.ssmidsOfInterestObjects[2]

In [ ]:
testObject.tracks['59581-59596'][0].diasources[0]

In [ ]:
testObject.tracks['59593-59608'][2].isSubset

In [ ]:
testObject.tracks['59593-59608'][2].isTrue

In [ ]:
def plotTrackObj(ax, track):
    ras = []
    decs = []
    for diasource in track.diasources:
        ras.append(diasource.ra)
        decs.append(diasource.dec)
        
    ax.plot(ras, decs)

In [ ]:
fig, ax = plt.subplots(1,1)
for track in testObject.tracks['59593-59608']:
    plotTrackObj(ax, track)

In [ ]:
def makeContiguous(angles):
    a0 = angles[0]
    output = [a0]
    for angle in angles[1:]:
        while abs(angle - a0) > 180:
            if angle > a0:
                angle -= 360.
            else:
                angle += 360.
        output.append(angle)
    return output

def convertToStandardDegrees(angle):
    while angle > 360.:
        angle -= 360.
    while angle < 0.:
        angle += 360.
    return angle

def calcDegToRad(angle):
    return angle*(np.pi/180.0)

def calcRadToDeg(angle):
    return angle*(180.0/np.pi)

def calcAngularDistance(a, b):
    while abs(a - b) > 180:
        if a > b:
            b += 360.
        else:
            a += 360.
    return a - b

def calcGreatCircleDistance(ra0, dec0, ra1, dec1):
    ra_dist = calcAngularDistance(ra0, ra1);
    dec_dist = calcAngularDistance(dec0, dec1);    
    # Convert all factors to radians
    ra_dist = calcDegToRad(convertToStandardDegrees(ra_dist));
    dec_dist = calcDegToRad(convertToStandardDegrees(dec_dist));
    dec0 = calcDegToRad(convertToStandardDegrees(dec0));
    dec1 = calcDegToRad(convertToStandardDegrees(dec1));
    r = 2*np.arcsin(np.sqrt((np.sin(dec_dist/2.))**2 + np.cos(dec0)*np.cos(dec1)*(np.sin(ra_dist/2))**2));
    # Back to degrees
    return calcRadToDeg(r);
    
def calcRMS(diasources):
    t0 = min(map(lambda x: x.mjd, diasources))
    ras = []
    decs = []
    mjds = []
    for diasource in diasources:
        ras.append(diasource.ra)
        decs.append(diasource.dec)
        mjds.append(diasource.mjd - t0)
    ras = makeContiguous(ras)
    decs = makeContiguous(decs)
    ras = np.array(ras)
    decs = np.array(decs)
    mjds = np.array(mjds)

    raFunc, raRes, rank, svd, rcond = np.polyfit(mjds, ras, 2, full=True)
    decFunc, decRes, rank, svd, rcond = np.polyfit(mjds, decs, 2, full=True)
    raFunc = np.poly1d(raFunc)
    decFunc = np.poly1d(decFunc)

    #now get the euclidean distance between predicted and observed for each point
    netSqDist = 0.0
    dists = []
    for i in range(len(mjds)):
        predRa = raFunc(mjds[i])
        predDec = decFunc(mjds[i])
        dist = calcGreatCircleDistance(predRa, predDec, ras[i], decs[i])
        dists.append(dist)
        if (dist > .1):
            print "Unexpected wierdness, diasource had angular distance of %f from best-fit curve prediction" % (dist)
            print "Predicted RA, Dec were ", predRa, predDec
            print "observed RA, Dec were ", ras[i], decs[i]
            print "all RAs were ", ras
            print "all decs were ", decs
        sqDist = dist**2
        #print "got euclidean distance was ", sqDist
        netSqDist += sqDist

    rms = np.sqrt(netSqDist / len(diasources))
    if (rms > .1):
        print "RMS error was %f " % (rms)
    return rms, raRes[0], decRes[0], dists

def countFindableTrueTrackletsAndSSMIDs(dataframe, minDetections, vmax):
    findableTrueTracklets = 0
    
    possible_ssmids = dataframe.groupby("ssmid").filter(lambda x: len(x) >= minDetections)
    unique_ssmids = possible_ssmids['ssmid'].unique()
    findable_ssmids = []
    
    for unique_ssmid in unique_ssmids:
        detections = possible_ssmids[possible_ssmids["ssmid"] == unique_ssmid]
        detections.sort(columns="mjd")

        start_mjd = min(detections['mjd'])
        end_mjd = max(detections['mjd'])

        dt = end_mjd - start_mjd
        max_distance = dt*vmax

        total_distance = 0.0
        total_time = 0.0

        for det0, det1 in zip(detections.iloc[:-1].itertuples(), detections.iloc[1:].itertuples()):
            total_distance += calcGreatCircleDistance(det0[3], det0[4], det1[3], det1[4])

        if max_distance > total_distance:
             findableTrueTracklets += 1
             findable_ssmids.append(unique_ssmid)
                
    return findableTrueTracklets, findable_ssmids

In [ ]:
42323.23 - 42323.20

In [ ]:
countFindableTrueTrackletsAndSSMIDs(objs, 2, 0.5)

In [ ]:
def countFindableTrueTracks(dataframe, minDetectionsPerNight, minNights):
    findableTracks = 0
    
    possible_ssmids = dataframe.groupby("ssmid").filter(lambda x: len(x) >= minDetectionsPerNight*minNights)
    unique_ssmids = possible_ssmids['ssmid'].unique()
    findable_ssmids = []
    
    for unique_ssmid in unique_ssmids:
        detections = possible_ssmids[possible_ssmids["ssmid"] == unique_ssmid]
        unique, counts = np.unique(detections.sort(columns="mjd")['mjd'].unique().astype(int), return_counts=True)
        if len(counts[counts >= minDetectionsPerNight]) >= minNights:
            findableTracks += 1
            findable_ssmids.append(unique_ssmid)

    return findableTracks, findable_ssmids

In [ ]:
countFindableTrueTracks(objs, 2, 3)

In [ ]:
len(objs.index)

In [ ]:
for unique_ssmid in unique_ssmids:
        detections = possible_ssmids[possible_ssmids["ssmid"] == unique_ssmid]
    

In [ ]:
detections.filter(int(detections['mjd']))

In [ ]:
int(analysis.windows[0].split('-')[1]) - int(analysis.windows[0].split('-')[0])

In [ ]:
text.astype(int)

In [ ]:
def checkWindow(window, onlyFullWindows, fullWindowSize):
    if onlyFullWindows:
        if int(window.split('-')[1]) - int(window.split('-')[0]) == fullWindowSize:
            return True
        else:
            return False
    else:
        return True
 

In [ ]:
checkWindow(analysis.windows[15], True, 15)

In [ ]:
a = np.array([])

In [ ]:
b = np.append(a, 23)

In [ ]:
np.append(b, 25)

In [ ]:
b.extend(3)

In [ ]:
np.empty([23])

In [ ]:
analysis.__dict__

In [ ]:
analysis.trackletFileSizes['59580']

In [ ]:
fig, ax = plt.subplots(1,1)

for i, night in enumerate(analysis.nights):
    ax.scatter(i, analysis.trackletFileSizes[night])
    ax.scatter(i, analysis.collapsedTrackletFileSizes[night])
    ax.scatter(i, analysis.purifiedTrackletFileSizes[night])
    ax.scatter(i, analysis.finalTrackletFileSizes[night])
    
for i, window in enumerate(analysis.windows)
    

In [ ]:
fig, ax = plt.subplots(1,1)
  
for i, window in enumerate(analysis.windows):
    ax.scatter(i, analysis.trackFileSizes[window], color='red')
    ax.scatter(i, analysis.trackDetFileSizes[window], color='blue')
    ax.scatter(i, analysis.trackIdFileSizes[window])
    

def plotAxisSetter(ax)
ax.set_ylim([0,15000]);
ax.set_xlim([0,30])
    

In [ ]:
import lsst.sims.maf.metrics

In [ ]:
import MopsReader

In [ ]:
import moMetrics

In [ ]:
from lsst.sims.maf import metrics